Command line tutorial
=====================

This tutorial will walk you through the usage of the ``lib5c`` command-line
tools.

Follow along in Google colab
-----------------

You can run and modify the cells in this notebook tutorial live using Google colaboratory by clicking the link below:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thomasgilgenast/lib5c-tutorials/blob/master/commandline-tutorial.ipynb)

To simply have all the cells run automatically, click `Runtime > Run all` in the colab toolbar.

Make sure `lib5c` is installed
------------------------------

Inside a fresh virtual environment, run

In [ ]:
!pip install lib5c

Make a directory and get data
-----------------------------

If you haven't completed the [pipeline tutorial](pipeline_tutorial.ipynb) yet,
make a directory for the tutorial:

```
$ mkdir lib5c-tutorial
$ cd lib5c-tutorial
```

and prepare the example data in `lib5c-tutorial/input` as shown in the
[pipeline tutorial](pipeline_tutorial.ipynb).

In [ ]:
!mkdir input
!wget -qO- 'http://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE68582&format=file&file=GSE68582%5FBED%5FES%2DNPC%2DiPS%2DLOCI%5Fmm9%2Ebed%2Egz' | gunzip -c > input/BED_ES-NPC-iPS-LOCI_mm9.bed
!wget -qO- 'http://www.ncbi.nlm.nih.gov/geo/download/?acc=GSM1974095&format=file&file=GSM1974095%5Fv65%5FRep1%2Ecounts%2Etxt%2Egz' | gunzip -c > input/v65_Rep1.counts
!wget -qO- 'http://www.ncbi.nlm.nih.gov/geo/download/?acc=GSM1974096&format=file&file=GSM1974096%5Fv65%5FRep2%2Ecounts%2Etxt%2Egz' | gunzip -c > input/v65_Rep2.counts
!wget -qO- 'http://www.ncbi.nlm.nih.gov/geo/download/?acc=GSM1974099&format=file&file=GSM1974099%5FpNPC%5FRep1%2Ecounts%2Etxt%2Egz' | gunzip -c > input/pNPC_Rep1.counts
!wget -qO- 'http://www.ncbi.nlm.nih.gov/geo/download/?acc=GSM1974100&format=file&file=GSM1974100%5FpNPC%5FRep2%2Ecounts%2Etxt%2Egz' | gunzip -c > input/pNPC_Rep2.counts
!sed -i '/Sox2\|Klf4/!d' input/BED_ES-NPC-iPS-LOCI_mm9.bed

Note for Docker image users
---------------------------

If you are using `lib5c` from the Docker image, run:

    $ docker run -it -v <full path to lib5c-tutorial>:/lib5c-tutorial creminslab/lib5c:latest
    root@<container_id>:/# cd /lib5c-tutorial

and continue running all tutorial commands in this shell.

Trimming low-quality primers
----------------------------

We will first trim away low-quality primers by running

In [ ]:
!lib5c trim -p input/BED_ES-NPC-iPS-LOCI_mm9.bed -t trimmed/%s.counts trimmed/primers_trimmed.bed 'input/*.counts'

Normalization
-------------

For this tutorial, we will first use Knight-Ruiz matrix balancing to correct our
data for primer effects by running

In [ ]:
!lib5c kr trimmed/pNPC_Rep2.counts kr/pNPC_Rep2.counts

`lib5c kr` specifies that the specific subcommand we want is `kr`, the
subcommand for Knight-Ruiz balancing. To see all the available subcommands, you
can run

In [ ]:
!lib5c -h

To see detailed help for the `kr` subcommand, you can run

In [ ]:
!lib5c kr -h

The two required positional command line arguments for `lib5c kr` are
`infile` (the countsfile we want to balance) and `outfile` (where to write
the balanced countsfile).

### Processing multiple input files

If we want to process a lot of countsfiles, we can use a pattern like this

In [ ]:
!lib5c kr 'trimmed/*.counts' kr/%s.counts

Here instead of a single `infile`, we have put in a glob-expandable pattern
that will match all the countsfiles in the `input` directory. We have to quote
this pattern to prevent the shell from expanding it. If we are processing
multiple input countsfiles, then we expect multiple countsfiles as the output.
To specify this easily, we include `%s` in the `outfile`, which will be
replaced with the replicate name (derived automatically from the input
filenames).

If you're running this in a cluster environment with the LSF (aka `bsub`) job
scheduler available, you can install the `bsub` Python package via

    $ pip install bsub

and then all the input files will be processed in parallel via `bsub`.

If you aren't using the LSF job scheduler, you can still specify multiple input
files, but they will be processed in series.

Plotting heatmaps
-----------------

Let's take a look at the countsfiles we just generated.

One way to visualize countsfiles is to plot heatmaps of them. To do this, we can
run

In [ ]:
!lib5c plot heatmap -p trimmed/primers_trimmed.bed --region Sox2 kr/pNPC_Rep2.counts kr/pNPC_Rep2_Sox2.png

The resulting image should look something like this

In [ ]:
from IPython.display import Image
Image(filename='kr/pNPC_Rep2_Sox2.png', width=400)

The `lib5c plot heatmap` subcommand is a lot like the `lib5c kr` subcommand.
In fact, most `lib5c` subcommands work in almost exactly the same way, so
learning to use one can help you understand the others.

### Specifying a primerfile/binfile with `-p/--primerfile`


Almost every `lib5c` subcommand that takes countsfiles as its input also
requires a primerfile or a bedfile that it uses to make sure the countsfiles get
parsed correctly. This is accomplished with the `-p` flag. We didn't have to
use the `-p` flag in the `lib5c kr` example above because the primerfile we
wanted to use was already in the same directory as the countsfiles, so `lib5c`
was able to find it automatically. In general though, we might be trying to
process countsfiles that are separated (directory-wise) from the primerfile that
describes their genomic loci, in which case we would have to use the `-p` flag
as shown above.

### Multiple output files (one per region)

Some `lib5c` subcommands generate multiple output files (typically one per region). To create
an output file for only one region, use the `--region` (shorter form `-r`)
as shown above. Alternatively, if we want to generate output for all regions, we
can skip the `-r/--region` flag and include a `%r` in the `outfile`
instead, which will be replaced with the region name. Combining this with the
"multiple input files" idea from above, we can write something like

In [ ]:
!lib5c plot heatmap -p trimmed/primers_trimmed.bed 'kr/*.counts' kr/%s_%r.png

Binning
-------

Let's bin our Knight-Ruiz balanced data!

Before we do this, we need to generate some bins to tile our 5C regions. To do
this, we run

In [ ]:
!lib5c determine-bins -w 4000 trimmed/primers_trimmed.bed bedfiles/4kb_bins.bed

The `-w/--bin_width` flag specifies the width of each bin in base pairs, and
`bedfiles/4kb_bins.bed` specifies the name of the newly-created bin bedfile.

Now that we have a bin bedfile describing the bins, we can figure out what the
counts values for each bin should be by running

In [ ]:
!lib5c bin -w 24000 -p trimmed/primers_trimmed.bed -b bedfiles/4kb_bins.bed kr/pNPC_Rep2.counts binned/pNPC_Rep2.counts

Notice that the `lib5c bin` subcommand needs both a primerfile (specified with
`-p`) and a bin bedfile (specified with `-b`). It needs the primerfile to
parse the input files, but it also needs a bin bedfile to know where the bins
should be and to write the output files. The last two arguments are just the
input file and the output file, just like in the previous subcommands.

We can plot a heatmap of our binned data by running

In [ ]:
!lib5c plot heatmap -p bedfiles/4kb_bins.bed -r Sox2 -R -g mm9 binned/pNPC_Rep2.counts binned/pNPC_Rep2_Sox2.png

The result should look something like this

In [ ]:
Image(filename='binned/pNPC_Rep2_Sox2.png', width=500)

Expected modeling
-----------------

In order to identify "enriched" interactions in our freshly-binned data, we
first need to answer the question: "Enriched with respect to what?" The answer
to this question is an expected model, which represents our understanding of
what we would expect the interaction frequency between any two bins to be. This
expected value depends both on the linear genomic separation between the bins,
as well as the effects of local contact domain structure.

We can generate an expected model by running

In [ ]:
!lib5c expected -p bedfiles/4kb_bins.bed -RED binned/pNPC_Rep2.counts expected/pNPC_Rep2.counts

Note that for consistency, the flag used to specify the bin bedfile
`bedfiles/4kb_bins.bed` is still `-p`. Most `lib5c` subcommands work on
both fragment-level and bin-level data, with essentially no changes to the
parameters.

We are using the `-R/--regression`, `-E/--exclude_near_diagonal` and
`-D/--donut_correction` flags to indicate that we want to start with a
regional expected model fitted with a simple log-log regression which ignores
points too close to the diagonal, and then apply donut correction to account for
local variations in contact domain structure.

Variance modeling
-----------------

Armed with an expected model, we can now readily identify which pairs of bins
interact more frequently than expected. However, what we really want to know is
which pairs of bins interact *significantly* more frequently than expected. In
order to understand the extent of insignificant, noise-driven variations at each
point, we need to construct a variance model - a statistical variance estimate
for each pair of bins.

We can generate a variance model by running

In [ ]:
!lib5c variance -p bedfiles/4kb_bins.bed binned/pNPC_Rep2.counts expected/pNPC_Rep2.counts variance/pNPC_Rep2.counts

By default, this will fit a log-normal deviation-based distance-variance relationship to
the observed data.

P-value calling
---------------

Finally, we will call p-values for the interactions in our dataset

In [ ]:
!lib5c pvalues -p bedfiles/4kb_bins.bed binned/pNPC_Rep2.counts expected/pNPC_Rep2.counts variance/pNPC_Rep2.counts -L norm pvalues/pNPC_Rep2.counts

Calling p-values simply parametrizes a distribution of a specified family (here
we chose `-L norm` for the lognormal distribution) with mean value taken from
the expected model and variance taken from the variance model, and then calls a
right-tail p-value for the observed value in the binned data.

### P-value heatmaps


To visualize p-values, we can use the `lib5c plot heatmap` subcommand as
above, but we need to invoke it like

In [ ]:
!lib5c plot heatmap -p bedfiles/4kb_bins.bed -r Sox2 -PR -g mm9 pvalues/pNPC_Rep2.counts pvalues/pNPC_Rep2_Sox2.png

Here we are using the `-P/--pvalue` flag to indicate that we are trying to
visualize p-values, the `-R/--rulers` flag to indicate that we want to add
genomic coordinate rulers to the heatmap, and the `-g/--genes` flag to
indicate that we want to add gene tracks from the mm9 reference genome.

The results should look something like this:

In [ ]:
Image(filename='pvalues/pNPC_Rep2_Sox2.png', width=500)

Next steps
----------

Go ahead and explore the other `lib5c` subcommands! Remember that you can list
all the subcommands and get detailed help for a particular subcommand with the
`-h/--help` flag.